In [1]:
from dotenv import load_dotenv

load_dotenv()

True

In [2]:
from langchain_openai import ChatOpenAI

llm = ChatOpenAI(model='gpt-4o')
small_llm = ChatOpenAI(model='gpt-4o-mini')


In [3]:
from typing_extensions import TypedDict
from langgraph.graph import StateGraph 


class AgentState(TypedDict):
    query: str
    context: list 
    answer: str 

graph_builder = StateGraph(AgentState)

In [4]:
from langchain_core.prompts import ChatPromptTemplate 
from pydantic import BaseModel, Field
from typing import Literal 


class Route(BaseModel):
    target: Literal['income_tax', 'llm', 'real_estate_tax'] = Field(
        description='the target for the query to answer'
    )

router_system_prompt = """
you are an expert at routing a user's questions to 'income_tax', 'llm', or 'real_estate_tax'.
'income_tax' contains information about income tax up to December 2024. 
'real_estate_tax' contains inforamtion about real estate tax up to December 2024.
if you think the question is not related to either 'income_tax' or 'real_estate_tax'
you can route it to 'llm'.
"""

route_prompt = ChatPromptTemplate.from_messages([
    ('system', router_system_prompt),
    ('human', '{query}')
])

structured_route_llm = small_llm.with_structured_output(Route)

def router(state: AgentState) -> Literal['income_tax', 'llm', 'real_estate_tax']:
    query = state['query']
    route_chain = route_prompt | structured_route_llm 
    route = route_chain.invoke(query)
    return route.target



In [5]:
from langchain_core.output_parsers import StrOutputParser

def call_llm(state: AgentState) -> AgentState:
    query = state['query']

    llm_chain = small_llm | StrOutputParser()
    llm_answer = llm_chain.invoke(query)
    return {'answer': llm_answer}



In [6]:
from income_tax_graph import graph as income_tax_agent
from real_estate_tax_graph import graph as real_estate_agent

graph_builder.add_node('income_tax', income_tax_agent)
graph_builder.add_node('real_estate_tax', real_estate_agent)
graph_builder.add_node('llm', call_llm)


/Users/robert/Desktop/langGrahpPrac/.venv/lib/python3.13/site-packages/langsmith/client.py:256: LangSmithMissingAPIKeyWarning: API key must be provided when using hosted LangSmith API
  warnings.warn(
/Users/robert/Desktop/langGrahpPrac/.venv/lib/python3.13/site-packages/langsmith/client.py:256: LangSmithMissingAPIKeyWarning: API key must be provided when using hosted LangSmith API
  warnings.warn(
/Users/robert/Desktop/langGrahpPrac/.venv/lib/python3.13/site-packages/langsmith/client.py:256: LangSmithMissingAPIKeyWarning: API key must be provided when using hosted LangSmith API
  warnings.warn(
/Users/robert/Desktop/langGrahpPrac/.venv/lib/python3.13/site-packages/langsmith/client.py:256: LangSmithMissingAPIKeyWarning: API key must be provided when using hosted LangSmith API
  warnings.warn(


In [7]:
from langgraph.graph import START, END

graph_builder.add_conditional_edges(
    START, 
    router,
    {
        'income_tax': 'income_tax',
        'real_estate_tax': 'real_estate_tax',
        'llm': 'llm'
    }
)

graph_builder.add_edge(
    'income_tax',
    END
)

graph_builder.add_edge(
    'real_estate_tax',
    END
)

graph_builder.add_edge(
    'llm',
    END
)

graph = graph_builder.compile()


In [8]:
graph.get_graph().print_ascii()

                   +-----------+                        
                   | __start__ |                        
                   +-----------+..                      
               ....       .       ...                   
            ...          .           ...                
          ..             .              ..              
+------------+       +-----+       +-----------------+  
| income_tax |       | llm |       | real_estate_tax |  
+------------+*      +-----+       +-----------------+  
               ****      *        ***                   
                   ***    *    ***                      
                      **  *  **                         
                    +---------+                         
                    | __end__ |                         
                    +---------+                         


In [9]:
initial_state = {'query': '소득세란 무엇인가요?'}
graph.invoke(initial_state)

{'query': '소득세란 무엇인가요?',
 'context': [Document(metadata={'source': './documents/income_tax.txt'}, page_content='| 1,400만원 초과     | 84만원 + (1,400만원을 초과하는 금액의 15퍼센트)                    |\n| 5,000만원 이하     | 624만원 + (5,000만원을 초과하는 금액의 24퍼센트)                  |\n| 8,800만원 이하     | 1,536만원 + (8,800만원을 초과하는 금액의 35퍼센트)                |\n| 1억5천만원 이하    | 3,706만원 + (1억5천만원을 초과하는 금액의 38퍼센트)                |\n| 3억원 이하        | 9,406만원 + (3억원을 초과하는 금액의 40퍼센트)                    |\n| 5억원 이하        | 1억7,406만원 + (5억원을 초과하는 금액의 42퍼센트)                 |\n| 10억원 이하       | 3억3,406만원 + (10억원을 초과하는 금액의 45퍼센트)                |\n법제처 35 국가법령정보센터\n소득세법\n② 거주자의 퇴직소득에 대한 소득세는 다음 각 호의 순서에 따라 계산한 금액(이하 "퇴직소득 산출세액"이라 한다)으로 한다. <개정 2013. 1. 1, 2014. 12. 23.>\n   1. 해당 과세기간의 퇴직소득과세표준에 제11항의 세율을 적용하여 계산한 금액\n   2. 제1호의 금액을 12로 나눈 금액에 근속연수를 곱한 금액\n   3. 삭제 <2014. 12. 23.>\n   [전문개정 2009. 12. 31.]\n제2관 세액공제 <개정 2009. 12. 31.>\n제56조(배당세액공제) ① 거주자의 종합소득금액에 제17조제3항 각 호 외의 부분 단서가 적용되는 배당소득금액이 합산되어 있는 경우에는 같은 항 각 호 외의 부분 단

In [10]:
initial_state = {
    'query': '종합소득세란 무엇인가요?'
}

graph.invoke(initial_state)

{'query': '종합소득세란 무엇인가요?',
 'context': [Document(metadata={'source': './documents/income_tax.txt'}, page_content='종합소득과세표준 = (종간예납기간의 종합소득금액 × 2) - 이월결손금 - 종합소득공제\n종합소득 산출세액 = 종합소득 과세표준 × 기본세율\n3.'),
  Document(metadata={'source': './documents/income_tax.txt'}, page_content='| 1,400만원 초과     | 84만원 + (1,400만원을 초과하는 금액의 15퍼센트)                    |\n| 5,000만원 이하     | 624만원 + (5,000만원을 초과하는 금액의 24퍼센트)                  |\n| 8,800만원 이하     | 1,536만원 + (8,800만원을 초과하는 금액의 35퍼센트)                |\n| 1억5천만원 이하    | 3,706만원 + (1억5천만원을 초과하는 금액의 38퍼센트)                |\n| 3억원 이하        | 9,406만원 + (3억원을 초과하는 금액의 40퍼센트)                    |\n| 5억원 이하        | 1억7,406만원 + (5억원을 초과하는 금액의 42퍼센트)                 |\n| 10억원 이하       | 3억3,406만원 + (10억원을 초과하는 금액의 45퍼센트)                |\n법제처 35 국가법령정보센터\n소득세법\n② 거주자의 퇴직소득에 대한 소득세는 다음 각 호의 순서에 따라 계산한 금액(이하 "퇴직소득 산출세액"이라 한다)으로 한다. <개정 2013. 1. 1, 2014. 12. 23.>\n   1. 해당 과세기간의 퇴직소득과세표준에 제11항의 세율을 적용하여 계산한 금액\n   2. 제1호의 금액을 12로 나눈 금액에 근속연수를 곱한 금액\n   3. 삭제

In [11]:
initial_state = {
    'query': '집 15억은 세금을 얼마나 내나요?'
}

graph.invoke(initial_state)

{'query': '집 15억은 세금을 얼마나 내나요?',
 'answer': '주어진 정보를 바탕으로 과세표준이 1.35억 원이고, 1세대 1주택자의 경우입니다. \n\n1세대 1주택자의 주택 수가 2주택 이하이므로, 이에 해당하는 세율을 적용합니다.\n\n과세표준이 3억 원 이하이므로, 세율은 1천분의 5입니다.\n\n따라서, 종합부동산세는 다음과 같이 계산됩니다:\n\n종합부동산세 = 과세표준 × 세율  \n종합부동산세 = 1.35억 원 × 0.005  \n종합부동산세 = 0.0675억 원\n\n0.0675억 원은 675만 원입니다.\n\n따라서, 종합부동산세는 675만 원입니다.'}

In [12]:
initial_state = {'query': '약수역에서 미팅하기 좋은 카페는 어디인가요?'}
graph.invoke(initial_state)

{'query': '약수역에서 미팅하기 좋은 카페는 어디인가요?',
 'answer': '약수역 근처에는 여러 카페가 있어 미팅하기 좋은 장소가 많이 있습니다. 몇 가지 추천 드리자면:\n\n1. **카페 드 파리** - 아늑한 분위기와 다양한 음료 메뉴가 있어 미팅하기에 적합합니다.\n2. **스타벅스 약수점** - 편안한 좌석과 안정적인 와이파이가 있어 업무 미팅에도 좋습니다.\n3. **이디야커피 약수역점** - 저렴한 가격에 커피를 즐길 수 있고, 넓은 공간이 있어 대화하기 좋습니다.\n4. **카페 마마스** - 브런치와 커피를 함께 즐길 수 있는 곳으로, 분위기가 좋습니다.\n\n각 카페의 분위기나 메뉴는 다를 수 있으니, 미팅의 성격에 맞춰 선택해보세요!'}